In [92]:
import pandas as pd
import numpy as np
import json
import requests as r
import time
from tqdm import tqdm

Temperature Query
```
SELECT
    wx.id,
    name,
    state,
    latitude,
    longitude,
    min(value/10) as min_high,
    APPROX_QUANTILES(value/10, 20) as quantiles_by_5,
    max(value/10) as max_high
FROM
    bigquery-public-data.ghcn_d.ghcnd_2020 AS wx
JOIN
    bigquery-public-data.ghcn_d.ghcnd_stations AS stn
ON
    wx.id = stn.id
WHERE
    wx.element = 'TMAX'
    AND wx.qflag IS NULL
    AND stn.state IS NOT NULL
GROUP BY wx.id, name, state, latitude, longitude
```

Precipitation Query
```
SELECT
    wx.id,
    name,
    state,
    latitude,
    longitude,
sum(value/10) AS total_precip_mm,
count(distinct case when value > 0 then `date` end) as num_precip_days,
count(distinct case when value/10 > 1 then `date` end) as num_precip_days_greater_1mm,
FROM
    bigquery-public-data.ghcn_d.ghcnd_2020 AS wx
JOIN
    bigquery-public-data.ghcn_d.ghcnd_stations AS stn
ON
    wx.id = stn.id
WHERE
    wx.element = 'TMAX'
    AND wx.qflag IS NULL
    AND stn.state IS NOT NULL
GROUP BY wx.id, name, state, latitude, longitude
```

In [110]:
precip = pd.read_json("data_sources/additional_sources/weather/2020_precipitation.json")
temp = pd.read_json("data_sources/additional_sources/weather/2020_temperature.json")

idToLatLon = pd.concat([precip[["id", "latitude", "longitude"]], temp[["id", "latitude", "longitude"]]]) \
    .drop_duplicates() \
    .sort_values(["latitude", "longitude"]) \
    .reset_index(drop = True)

In [111]:
idToLatLon.head()

,id,latitude,longitude
0,AYW00090001,-90.000,0.000
1,AYM00089606,-78.450,106.867
2,AYM00089034,-77.867,-34.617
3,AYM00089022,-75.450,-26.217
4,AYM00089662,-74.700,164.100


In [116]:
# using fcc area api
def latLonRequest(lat, lon): 
    result = r.request(
        'get', 
        'https://geo.fcc.gov/api/census/block/find', 
        params={'latitude':lat, 'longitude': lon, 'censusYear': 2020, 'format': 'json'})
    return json.loads(result.text)

def latLonToCounty(row, lat_column="latitude", lon_column="longitude"): 
    block_response = latLonRequest(row[lat_column], row[lon_column])
    block_response = block_response if block_response is not None else {}
    row["county"] = block_response.get("County", {}).get("name")
    row["county_fips"] = block_response.get("County", {}).get("FIPS")
    row["state_id"] = block_response.get("State", {}).get("code")
    row["state_name"] = block_response.get("State", {}).get("name")
    row["state_fips"] = block_response.get("State", {}).get("FIPS")
    row["api_request_status"] = block_response.get("status")
    return row

In [117]:
latLonRequest(45.0900, -71.2906)

{'Block': {'FIPS': '330079501002038',
  'bbox': [-71.303427, 45.087563, -71.287331, 45.09677]},
 'County': {'FIPS': '33007', 'name': 'Coos'},
 'State': {'FIPS': '33', 'code': 'NH', 'name': 'New Hampshire'},
 'status': 'OK',
 'executionTime': '0'}

In [118]:
# calls need delay since api docs specify need to space requests
def latLonToCountyRequestWithDelay(df, lat_column, lon_column, delay_sec=0.5):
    for (index, data) in tqdm(df.iterrows()):
        time.sleep(delay_sec)
        yield latLonToCounty(data, lat_column, lon_column)

In [121]:
for i in range(7, (idToLatLon.shape[0] // 1000) + 1): 
    print("part {}".format(i))
    latLonCounties = list(latLonToCountyRequestWithDelay(idToLatLon[i*1000:(i+1)*1000], "latitude", "longitude", 0.5))
    pd.DataFrame(latLonCounties) \
        .to_csv("data_sources/additional_sources/weather/2020_weather_station_to_county_{part}.csv".format(part=i), 
                index=False)

0it [00:00, ?it/s]

part 7


1000it [14:52,  1.12it/s]
0it [00:00, ?it/s]

part 8


1000it [15:06,  1.10it/s]
0it [00:00, ?it/s]

...

1000it [15:05,  1.10it/s]
0it [00:00, ?it/s]

part 12


1000it [15:16,  1.09it/s]
0it [00:00, ?it/s]

part 13


695it [10:29,  1.10it/s]


In [99]:
idToLatLon.shape[0]

13695

In [122]:
df_concated = pd.concat([
    pd.read_csv("data_sources/additional_sources/weather/2020_weather_station_to_county_{part}.csv".format(part=i)) 
    for i 
    in range(14)])

In [138]:
# write out full dataset
df_with_county = df_concated[~df_concated["county"].isna()]

# merge precip, temp, and county dfs
df_merged = precip.drop(columns=["latitude", "longitude", "state"]) \
    .merge(
        temp.drop(columns=["latitude", "longitude", "name", "state"]),
        how="inner",
        on="id"
    ) \
    .merge(
        df_with_county,
        how="inner",
        on="id"
    )

df_merged.to_csv("data_sources/additional_sources/2020_weather_station_to_county_complete.csv", index=False)

In [140]:
df_merged.head()

,id,name,total_precip_mm,num_precip_days,num_precip_days_greater_1mm,min_high,quantiles_by_5,max_high,latitude,longitude,county,county_fips,state_id,state_name,state_fips,api_request_status
0,USC00272999,FIRST CONNECTICUT LAKE,3976.7,288,282,-18.3,"[-18.3, -6.1, -3.3, -1.7, 0.0, 1.7, 2.8, 3.9, ...",32.8,45.0900,-71.2906,Coos,33007.0,NH,New Hampshire,33.0,OK
1,USW00003757,ST INIGOES WEBSTER NOLF,7932.0,363,363,3.3,"[3.3, 7.8, 10.6, 12.2, 13.3, 15.0, 16.7, 18.3,...",37.8,38.1417,-76.4292,St. Mary's,24037.0,MD,Maryland,24.0,OK
2,USC00164700,JENNINGS,8296.9,336,336,6.7,"[6.7, 11.7, 14.4, 17.8, 18.9, 20.6, 21.7, 22.8...",36.1,30.2003,-92.6642,Jefferson Davis,22053.0,LA,Louisiana,22.0,OK
3,USC00068138,STORRS,5641.6,347,343,-5.0,"[-5.0, 0.0, 2.2, 4.4, 5.6, 7.2, 8.3, 10.0, 12....",32.2,41.7950,-72.2286,Tolland,9013.0,CT,Connecticut,9.0,OK
4,USC00190998,BUFFUMVILLE LAKE,5906.3,349,344,-6.1,"[-6.1, 0.6, 2.8, 4.4, 5.6, 7.8, 8.9, 10.0, 12....",35.0,42.1164,-71.9075,Worcester,25027.0,MA,Massachusetts,25.0,OK


In [134]:
df_with_county.api_request_status.unique() # all requests returned 200 response (aka "ok")

array(['OK'], dtype=object)

In [158]:
def get_percentile(_20quantiles, percentile):
    return float(_20quantiles[int(percentile/0.05)])

for pct in [0.05, 0.25, 0.5, 0.75, 0.95]:
    df_merged["{:.2f}_percentile_high".format(pct)] = df_merged.quantiles_by_5.apply(lambda x: get_percentile(x, pct))

df_merged
df_aggregated_by_county = df_merged.groupby(["county", "county_fips", "state_id", "state_name", "state_fips"]).agg(
    {"total_precip_mm": lambda x: int(np.median(x)),
     "num_precip_days": lambda x: int(np.median(x)),
     "num_precip_days_greater_1mm": lambda x: int(np.median(x)),
     "0.05_percentile_high": np.median, 
     "0.25_percentile_high": np.median, 
     "0.50_percentile_high": np.median, 
     "0.75_percentile_high": np.median, 
     "0.95_percentile_high": np.median,
     "min_high": np.min,
     "max_high": np.max}
).reset_index()

In [159]:
df_aggregated_by_county.to_csv("data_sources/additional_sources/2020_weather_station_to_county_aggregated_by_county.csv", index=False)